In [1]:
## downloading corpus

In [2]:
import keras
import numpy as np

2024-07-01 12:53:33.757620: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-01 12:53:34.279851: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-07-01 12:53:34.279882: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-07-01 12:53:34.368095: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-01 12:53:35.521927: W tensorflow/stream_executor/platform/de

In [3]:
path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt'
)

In [4]:
text = open(path).read().lower()
print('Corpus length : ', len(text))

Corpus length :  600893


In [5]:
## Vectorization of sequences

In [6]:
maxlen = 60
step = 6
sentences = []
next_char = []

In [7]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_char.append(text[i + maxlen])

In [8]:
chars = sorted(list(set(text)))

In [9]:
char_indices = dict((char, chars.index(char)) for char in chars)

In [10]:
x = np.zeros(shape = (len(sentences), maxlen, len(chars)))

In [11]:
y = np.zeros(shape = (len(next_char), len(chars)))

In [12]:
for i, sentence in enumerate(sentences):
    for j, ch in enumerate(chars):
        x[i, j, char_indices[ch]] = 1
    y[i, char_indices[next_char[i]]] = 1

In [13]:
from keras import layers, models, optimizers

In [14]:
model = models.Sequential()
model.add(layers.LSTM(128, input_shape = (maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation = 'softmax'))

2024-07-01 12:53:39.529559: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-07-01 12:53:39.529905: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-07-01 12:53:39.529936: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mukesh-ThinkPad-E14): /proc/driver/nvidia/version does not exist
2024-07-01 12:53:39.530672: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               95232     
                                                                 
 dense (Dense)               (None, 57)                7353      
                                                                 
Total params: 102,585
Trainable params: 102,585
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(
    optimizer = optimizers.RMSprop(learning_rate = 0.001),
    loss = 'categorical_crossentropy'
)

In [17]:
import random
import sys

In [18]:
def sample(preds, temperature = 1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probs = np.randoom.multinomial(1, preds, 1)
    return np.argmax(probs)

In [19]:
for epoch in range(1, 60):
    print("Epoch : ", epoch)
    model.fit(x, y, batch_size = 128, epochs = 1)
    start_index = np.random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index : start_index + maxlen]
    print("-------------- Generated Text Seed------------------")
    print(generated_text)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print(f"----------------- temperature -------------{temperature}")
        sys.stdout.write(generated_text)
        for i in range(400):
            sample = np.zeros(shape = (1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sample[0, t, char_indices[char]] = 1
            preds = model.predict(sample, verbose = 0)[0]
            print(preds)
            print(temperature)
            next_index = sample(preds, temperature = temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)

Epoch :  1


2024-07-01 12:53:41.100028: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1369901520 exceeds 10% of free system memory.


783/783 [==============================] - 71s 88ms/step - loss: 3.0368
-------------- Generated Text Seed------------------
ey follow
one another with lightning rapidity are no longer 
----------------- temperature -------------0.2
ey follow
one another with lightning rapidity are no longer [1.83370076e-02 1.22712776e-01 8.57344188e-04 2.76150228e-03
 5.43052971e-04 3.15491052e-04 4.07504966e-04 1.64989009e-02
 6.43500779e-03 6.10851916e-03 4.19005737e-05 5.86373906e-04
 5.18426474e-04 1.20660952e-04 4.92285071e-05 1.03191815e-05
 4.96331413e-05 2.56419662e-05 2.28384524e-05 7.09000369e-06
 1.05001591e-03 1.00798393e-03 3.52105446e-04 5.51353674e-04
 2.08067886e-05 1.03721635e-04 2.37273835e-05 5.57050593e-02
 1.11897970e-02 2.35532243e-02 2.97662821e-02 9.37712789e-02
 1.99176297e-02 1.34356506e-02 4.89690565e-02 7.46538565e-02
 1.17378728e-03 3.34979314e-03 3.44769210e-02 1.94076709e-02
 5.94344772e-02 6.71132728e-02 2.04725042e-02 1.24061736e-03
 4.78826165e-02 6.14735112e-02 7.031

TypeError: 'numpy.ndarray' object is not callable